# Tutorial Content - alt

This notebook uses ITS_LIVE data downloaded from NSIDC access point

This notebook will walk you through steps to read in and organize velocity data in a raster format using xarray and rioxarray tools

First, lets install the python libraries that were listed on the [Software](software.ipynb) page:

In [ ]:
import geopandas as gpd
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
import xarray as xr
import numpy as np
import pandas as pd
import packaging
import pyproj

In [ ]:
gen_path = '/Users/emmamarshall/Desktop/phd_research/siparcs/'

In [ ]:
#data
itslive = rxr.open_rasterio('/Users/emmamarshall/Desktop/phd_research/siparcs/HMA_G0120_0000.nc').squeeze()


In [ ]:
itslive.v


In [ ]:
itslive.rio.crs.from_wkt

In [ ]:
type(itslive)
itslive.rio.crs

In [ ]:
#read in vector data 
se_asia = gpd.read_file('/Users/emmamarshall/Downloads/15rgi60SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')
#sw_asia = gpd.read_file('/Users/emmamarshall/Desktop/phd_research/nisar_prepwork/rgi_1km/sw_asia_1km.shp')
#c_asia = gpd.read_file('/Users/emmamarshall/Desktop/phd_research/nisar_prepwork/rgi_1km/central_asia_1km.shp') 

#se_asia_path = '/Users/emmamarshall/Desktop/phd_research/nisar_prepwork/rgi_1km/se_asia_1km.shp'
#sw_asia_path = '/Users/emmamarshall/Desktop/phd_research/nisar_prepwork/rgi_1km/sw_asia_1km.shp'
#c_asia_path = '/Users/emmamarshall/Desktop/phd_research/nisar_prepwork/rgi_1km/central_asia_1km.shp'




In [ ]:
se_asia_prj = se_asia.to_crs('+proj=lcc +lat_1=15 +lat_2=65 +lat_0=30 +lon_0=95 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs'
                             )

In [ ]:
se_asia_prj.plot()

In [ ]:
fig, ax = plt.subplots()

itslive.v.plot.imshow(ax=ax, alpha = 0.1)
se_asia_prj.plot(ax=ax, color='red')


In [ ]:
se_asia_prj['Unique_ID'] = se_asia_prj.index.astype(int)
se_asia_prj

In [ ]:
def rasterize_vector(gpdf_prj, raster_obj):  #for now, project objects outside of fn
    
    #use index as a unique key for each glacier
    gpdf_prj['Integer_ID'] = gpdf_prj.index.astype(int)
    #print(gpdf_utm['Integer_ID'])
    
    #rasterize glacier vector by unique id 

    out_grid = make_geocube(
            vector_data = gpdf_prj,
            measurements = ['Integer_ID'],
            like = raster_obj['v'] #need to specify a var here, not sure best way to do that
            )
    
    #now merge the rasterized vector and the original raster togehter into a geocube
    out_grid['speed'] = (raster_obj.dims, raster_obj.v.values, raster_obj.attrs, raster_obj.encoding)
    
    #now, get velocity statistics of each 'region' (integer) using the mask
    #grouped_ID = out_grid.drop('spatial_ref').groupby(out_grid.Integer_ID)

    #compute zonal stats groupedd by ID
    #grid_mean_sp = grouped_ID.mean().rename({'speed': 'speed_mean'})
    #grid_min_sp = grouped_ID.min().rename({'speed': 'speed_min'})
    #grid_max_sp = grouped_ID.max().rename({'speed': 'speed_max'})
    #grid_std_sp = grouped_ID.max().rename({'speed': 'speed_std'})
    
    #merge each zonal stat xr obj into a single xr obj, convert to pandas df
    #zonal_stats = xr.merge([grid_mean_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
    #zonal_stats = zonal_stats.reset_index()
    
   # return zonal_stats
    return out_grid

In [ ]:
outgrid_seasia = rasterize_vector(se_asia_prj, itslive)


In [ ]:
outgrid_seasia

In [ ]:
outgrid_seasia.Integer_ID.plot.imshow()

In [ ]:
grouped_ID = outgrid_seasia.drop('spatial_ref').groupby(outgrid_seasia.Integer_ID)

In [ ]:
grid_mean_sp = grouped_ID.mean().rename({'speed': 'speed_mean'})
grid_median_sp = grouped_ID.median().rename({'speed': 'speed_median'})
grid_min_sp = grouped_ID.min().rename({'speed': 'speed_min'})
grid_max_sp = grouped_ID.max().rename({'speed': 'speed_max'})
#grid_std_sp = grouped_ID.max().rename({'speed': 'speed_std'})

In [ ]:
zonal_stats = xr.merge([grid_mean_sp, grid_median_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
zonal_stats = zonal_stats.reset_index()
zonal_stats

In [ ]:
#now, trying to merge zonal stats df back with original glacier df on integer_ID col
se_asia_glacier_data = se_asia_prj.merge(zonal_stats, on='Integer_ID')


In [ ]:
zonal_stats['speed_mean']

fig, ax = plt.subplots()
se_asia_glacier_data.plot.scatter(x='Integer_ID',y = 'speed_mean', c = 'darkblue', ax=ax)


In [ ]:
zonal_stats['speed_mean'].min()

In [ ]:
se_asia_glacier_data.plot(column='speed_mean', legend=True)